Install dash, visdcc and related components

In [1]:
!pip install dash
!pip install dash_core_components
!pip install dash_html_components
!pip install jupyter-dash
!pip install dash-renderer
!pip install visdcc
!pip install plotly==4.8

Import libraries

In [2]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State

import visdcc

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
print(plt.style.available)
mpl.style.use(['ggplot'])
import plotly.express as px

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


Load data

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/ibikinoradio/graph_visual/main/expenditures_zen_ts.csv')
df['Timestamp'] = df['Timestamp'].astype('datetime64[ns]')
df

,Timestamp,Grains,Seafood,Meat,Milk and eggs,Vegetables and seaweed,Fruits,Oils and seasonings,Confectionery,Cooked food,Beverages,Alcoholic beverages,Eating out,Rent,Equipment repair and maintenance,Electricity,Gas,Other light and heating,Water and sewage,Household durable goods,Interior equipment and decorations,Bedding,Housework miscellaneous goods,Housework consumables,Housework service,Japanese clothes,Western clothes,Shirts and sweaters,Underwear,Fabric and threads,Other clothing,Footwear,Clothing related services,Pharmaceuticals,Ingestion for maintaining health,Health and medical supplies and equipment,Health care service,Transportation,Car related expenses,Communication,"Tuition fees, etc.",Textbooks and learning reference materials,Supplementary education,Durable goods for culture and entertainment,Art and entertainment products,Books and other printed materials,Art and entertainment services,Miscellaneous expenses,Pocket money (unknown use),Entertainment expenses,Remittances
0,2000-01-01,5942,8143,6108,3372,7813,2524,2804,4553,7102,2718,2652,13132,11216,5342,11096,7008,2118,4733,2955,904,866,2070,1557,889,997,8314,3538,1408,279,1331,1615,885,1811,527,2536,5875,6589,15407,9235,9406,278,2843,3956,6124,4244,15296,18593,21858,42547,6512
1,2000-02-01,6809,8386,6394,3623,8477,2888,3072,5051,7278,2935,3007,10952,11266,7189,10973,7456,2639,4609,2605,877,709,1921,1770,837,876,5824,2423,1222,262,1038,1199,829,1981,535,2276,6887,5526,16957,8484,11411,219,2847,3029,5431,4540,15000,18408,20078,24030,7625
2,2000-03-01,7310,8967,6426,3900,9105,2959,3209,5321,7825,3320,3448,12236,11473,6926,10716,7591,2213,4812,3360,816,1125,2307,1960,860,245,8174,3086,1221,303,1135,2099,1165,2120,734,2359,6448,7014,23012,8934,12545,1095,4058,4367,7577,4711,17695,22057,21286,31723,11996
3,2000-04-01,7321,8434,6388,3766,9040,2754,3089,4840,7754,3390,3525,11856,11047,7769,10109,7256,1194,4349,2201,781,667,2312,2051,947,667,6465,3352,1266,264,1184,1975,1860,1926,505,2115,6607,7896,24604,8561,19239,917,3885,2987,7702,4484,17208,20016,19901,27505,19347
4,2000-05-01,7294,8516,6576,3878,9287,2897,3257,4994,7660,4023,3896,13124,11688,7556,8890,6608,719,4858,3070,1271,780,2292,2270,1002,627,5917,3758,1541,346,1287,1850,1957,1962,524,2276,6477,6918,19392,9580,8516,427,2568,2648,7476,4473,17802,18958,20030,27694,7156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2021-01-01,5988,5906,8031,3885,8712,2973,3844,6103,10764,4247,3253,7992,8404,7043,11875,5740,2568,5214,3815,615,881,2348,2959,654,85,3741,1377,912,122,828,1101,273,2464,1046,2950,6475,2227,21008,13585,5873,123,2770,2468,5686,3241,10313,23772,7395,19244,4867
253,2021-02-01,5847,5601,7406,3805,8320,2999,3673,6424,10468,4197,3128,7961,7980,7156,12854,5980,2198,5558,2871,468,674,2025,2830,685,59,2947,1157,647,120,479,910,387,2565,1024,2553,7010,2173,21601,12589,9328,147,2100,1911,5691,3128,9994,19993,6743,11377,4710
254,2021-03-01,6473,6097,7758,4146,8817,2973,3824,6801,10991,4735,3353,9693,9518,12118,13197,6425,1883,5894,3731,757,826,2472,3196,687,25,4597,1761,758,152,640,1448,610,2782,1174,2912,8153,3910,26917,14846,9556,748,3417,3841,7210,3667,12345,27213,6950,18742,9060
255,2021-04-01,6480,5733,7649,4060,8795,2861,3745,6162,10693,4798,3527,8594,8970,10568,10696,5674,1110,5318,3172,446,590,2329,3317,814,238,3375,1946,863,122,738,1258,745,2645,1166,2649,7476,4485,28105,13236,19239,641,2553,2663,7570,3474,12365,25207,6913,13893,11380


Function for getting a portion of data for 1 timestep

In [5]:
def get_data_for_timestamp(df_data, timestamp):

  # filter data
  ts_start = timestamp - pd.offsets.DateOffset(years=1)
  data = df_data[(df_data['Timestamp']<=timestamp)&(df_data['Timestamp']>=ts_start)]

  # make correlation matrix
  corr_matrix = data.corr()

  # make source-target data
  source_target = pd.DataFrame(columns=['Source', 'Target', 'Correlation'])

  for index, row in corr_matrix.iterrows():
    df_temp = pd.DataFrame()
    col_idx = corr_matrix.columns.get_loc(index)
    df_temp['Correlation'] = row[0:col_idx]
    df_temp['Target'] = df_temp.index
    df_temp.reset_index(drop=True, inplace=True)
    df_temp['Source'] = index

    source_target = pd.concat([source_target, df_temp], ignore_index=True)

  return source_target, corr_matrix

Make edges and nodes for the network graph

In [6]:
def get_nodes_edges(df):

  node_list = list(set(df['Source'].unique().tolist() + df['Target'].unique().tolist()))
  nodes = [{'id': node_name, 'label': node_name, 'shape': 'dot', 'size': 7} for i, node_name in enumerate(node_list)]

  edges = []
  for row in df.to_dict(orient='records'):
    source, target = row['Source'], row['Target']
    corr = row['Correlation']

    if corr > 0:
      color = "#0000ff"
    else:
      color = "#ff0000"

    edges.append({
          'id': source + "__" + target,
          'label' : "{:0.2f}".format(corr),
          'from': source,
          'to': target,
          'width': 1,
          'color':{ 'color': color},
          "smooth": False,
    })
  return nodes, edges

Functions for date slider (need to convert dates there and back, because slider doesn't support datetime)

In [7]:
import time

def unixTimeMillis(dt):
    ''' Convert datetime to unix timestamp '''
    return int(time.mktime(dt.timetuple()))

def unixToDatetime(unix):
    ''' Convert unix timestamp to datetime. '''
    return pd.to_datetime(unix,unit='s')

def getMarks(start, end, Nth=12):

    daterange = pd.date_range(start=start,end=end,freq='MS')

    ''' Returns the marks for labeling. 
        Every Nth value will be used.
    '''

    result = {}
    for i, date in enumerate(daterange):
        if(i%Nth == 1):
            # Append value to dict
            result[unixTimeMillis(date)] = str(date.strftime('%Y-%m'))
            
    return result

Functions for updating graphs


In [8]:
import plotly.express as px

def corr_heatmap(data):
  return px.imshow(data, title='Correlation heatmap for selected timestamp')

def timeline_plot(data):
  return px.line(data, x='Timestamp', y=data.columns[1:], title='Timeline')

Functions for tree-length calculation

In [9]:
def calc_tree_length(data):
  tree_len = (1 - data['Correlation'].abs()).sum()
  tree_aver = tree_len / len(data)
  return tree_len, tree_aver

Make dash app

In [10]:
# init some data for network
source_target, corr_matrix = get_data_for_timestamp(df, df['Timestamp'].max())
nodes, edges = get_nodes_edges(source_target)

# init app
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# App layout
app.layout = html.Div([html.Br(),
                       
                       html.H1('DATA CORRELATION DASHBOARD', 
                                style={'textAlign': 'center', 'color': '#454545',
                                'font-size': 30}),
                       
                       html.Br(),

                       html.Div(
                           style={'marginLeft':'1.5%', 'marginRight':'1.5%', 'marginBottom':'0.5%'},
                                children=[
                                          html.Div(
                                              style={'marginRight':'0.8%', 'width':'30%', 'backgroundColor':'#f1f1f1', 'display':'inline-block'},
                                              children=[html.P(style={'textAlign':'center', 'fontWeight':'bold', 'color':'#503D36', 'padding':'1rem'},
                                                               children='CURRENT TIMESTAMP'),
                                                        html.H3(id='timestamp_setting_output',
                                                            style={'textAlign':'center', 'fontWeight':'bold', 'color':'#454545'},
                                                            children='2021-05'
                                                                )]),
                                          html.Div(
                                              style={'marginRight':'0.8%', 'width':'30%', 'backgroundColor':'#f1f1f1', 'display':'inline-block'},
                                              children=[html.P(style={'textAlign':'center', 'fontWeight':'bold', 'color':'#503D36', 'padding':'1rem'},
                                                               children='FULL TREE LENGTH / AVERAGE'),
                                                        html.H3(id='tree_length_output',
                                                            style={'textAlign':'center', 'fontWeight':'bold', 'color':'#454545'},
                                                            children='0 / 0'
                                                                )]),
                                          html.Div(
                                              style={'marginTop':'0.5%', 'width':'30%', 'backgroundColor':'#f1f1f1', 'display':'inline-block'},
                                              children=[html.P(style={'textAlign':'center', 'fontWeight':'bold', 'color':'#503D36', 'padding':'1rem'},
                                                               children='LENGHT of SELECTED BRANCHES / AVERAGE'),
                                                        html.H3(id='branch_length_output',
                                                            style={'textAlign':'center', 'fontWeight':'bold', 'color':'#454545'},
                                                            children='0 / 0'
                                                                )]),
                                          ]),
                       
                       html.Br(), 

                       html.Div([dcc.Slider(id='time_slider',
                                            min = unixTimeMillis(df['Timestamp'].min()+pd.offsets.DateOffset(years=1)),
                                            max = unixTimeMillis(df['Timestamp'].max()),
                                            value = unixTimeMillis(df['Timestamp'].max()),
                                            marks=getMarks(df['Timestamp'].min()+pd.offsets.DateOffset(years=1), df['Timestamp'].max()),
                                            included=False,
                                            ),
                                 ], style={'textAlign': 'center'}),                       

                       html.Div(visdcc.Network(id = 'net', 
                                    data = {'nodes': nodes, 'edges': edges},
                                    options = dict(height= '600px', width= '100%',
                                                   physics={'solver': 'forceAtlas2Based',
                                                            'forceAtlas2Based': {'gravitationalConstant':-200}
                                                            })
                                    )),
                       
                       html.Br(),
                       
                       html.Div(["CORRELATION RANGE SETTINGS ",
                                 html.Br(),
                                 dcc.Input(id="corr_neg", type="number", debounce=True, value=0, min=-1, max=0, step=0.05),
                                 dcc.Input(id="corr_pos", type="number", debounce=True, value=0, min=0, max=1, step=0.05)],
                                style={'textAlign': 'center'}),
                       
                       html.Br(),
                       
                       html.Div(dcc.Checklist(id = 'check_filter',
                                              options=[{'label': 'Select All', 'value': 'Select All'}] + \
                                                      [{'label': node_name, 'value': node_name} for i, node_name in enumerate(df.columns[1:])],
                                              value=['Select All'] + list(df.columns[1:].values),
                                              labelStyle={'display': 'inline-block'})),
                       
                      html.Div([html.Div(dcc.Graph(id='correlation_plot')),
                                html.Div(dcc.Graph(id='timeline_plot'))
                                ]),
])

@app.callback(
              [Output(component_id='timestamp_setting_output', component_property='children'),
               Output(component_id='tree_length_output', component_property='children'),
               Output(component_id='branch_length_output', component_property='children'),
               Output(component_id='net', component_property='data'),
               Output(component_id='correlation_plot', component_property='figure'),
               Output(component_id='timeline_plot', component_property='figure'),
               Output(component_id='check_filter', component_property='value'),
               
              ],
              
              [Input(component_id='time_slider', component_property='value'),
               Input(component_id='check_filter', component_property='value'),
               Input(component_id='corr_neg', component_property='value'),
               Input(component_id='corr_pos', component_property='value')
               ]
              )

def update_output(timestamp, checkbox_values, negative_corr_limit, positive_corr_limit):

  #select all  logic
  if ('Select All' in checkbox_values) and (checkbox_values!=['Select All'] + list(df.columns[1:].values)):
    checkbox_values = ['Select All'] + list(df.columns[1:].values)
  
  # deselect all logic
  if checkbox_values == list(df.columns[1:].values):
    checkbox_values = []

  ['Select All'] + list(df.columns[1:].values)

  # get new dataframe for a timestamp
  source_target, corr_matrix = get_data_for_timestamp(df, unixToDatetime(timestamp) - pd.offsets.MonthBegin(1))

  # calc full tree length before filter
  full_length, full_average = calc_tree_length(source_target)

  # Fitler data by correlation limits
  source_target = source_target[((source_target['Correlation']>=positive_corr_limit) | \
                                 (source_target['Correlation']<=negative_corr_limit))]

  #Filter data by selected checkboxes
  source_target = source_target[(source_target['Source'].isin(checkbox_values)) & \
                                (source_target['Target'].isin(checkbox_values))]

  # calc length of selected branches
  branch_length, branch_average = calc_tree_length(source_target)

  # make nodes and edges
  nodes, edges = get_nodes_edges(source_target)

  # make graphs
  corr_plot = corr_heatmap(corr_matrix)
  time_plot = timeline_plot(df)

  update_result = ['{}'.format(unixToDatetime(timestamp).to_numpy().astype('datetime64[M]')),
                   '{:0.2f} / {:0.2f}'.format(full_length, full_average),
                   '{:0.2f} / {:0.2f}'.format(branch_length, branch_average),
                  {'nodes': nodes, 'edges': edges},
                   corr_plot,
                   time_plot,
                   checkbox_values,
  ]

  return update_result


# Run the app
if __name__ == '__main__':
    app.run_server(mode="external", host="localhost", port=7645, debug=True)

Dash app running on:


<IPython.core.display.Javascript object>